Driving Styles

In [11]:
import fastf1
import fastf1.plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Machine learning imports
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [4]:
fastf1.Cache.enable_cache('C:/Users/almas/OneDrive/Desktop/Purdue/Projects/F1/cache')  
year = 2025
track = 'Monza'

In [5]:
session = fastf1.get_session(year, track, 'R')
session.load()

core           INFO 	Loading data for Italian Grand Prix - Race [v3.6.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

In [10]:
drivers = session.results['Abbreviation'].unique()
print(drivers)

['VER' 'NOR' 'PIA' 'LEC' 'RUS' 'HAM' 'ALB' 'BOR' 'ANT' 'HAD' 'SAI' 'BEA'
 'TSU' 'LAW' 'OCO' 'GAS' 'COL' 'STR' 'ALO' 'HUL']


In [12]:
def get_lap_data(lap):
    tel = lap.get_telemetry().add_distance()
    if tel.empty:
        return None
    return {
        'Distance': tel['Distance'].values,
        'Speed': tel['Speed'].values,
        'Throttle': tel['Throttle'].mean(),
        'Brake': tel['Brake'].mean(),
        'Steering': tel['Steering'].values,
    }

In [ ]:
for driver in drivers:
    laps = session.laps.pick_driver(driver).pick_fastest()
    

C:\Users\almas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fastf1\core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


KeyError: 'Steering'